In [1]:
import numpy as np 
import pandas as pd
import sys
# !{sys.executable} -m pip install spotipy
import spotipy
import os
import spotipy.oauth2 as Auth 

CLIENT_ID = '60ddd115f669473890f90e8f1443e44c'
CLIENT_SECRET = '7861b9c48f4b4be0ad43a7ccf7b21a25'

spotifyAuth = Auth.SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
token = spotifyAuth.get_access_token()
spotify = spotipy.Spotify(auth=token)


In [2]:
def get_track_ids(filename):
    DATA = pd.read_csv(filename, skiprows=0, header=1)
    data = np.asarray(DATA)
    track_links = list(data[:, 0])
    track_ids = [link.split(':')[-1] for link in track_links]
    return track_ids

def get_pop_track_ids(filename):
    DATA = pd.read_csv(filename, skiprows=0, header=1)
    data = np.asarray(DATA)
    track_links = list(data[:, -1])
    track_ids = [link.split('/')[-1] for link in track_links]
    return track_ids

def order_data(filename):
    rows = set()
    csv_data = pd.read_csv(filename, skiprows=0, header=0)
#     csv_data.rename(columns = {'popularity':'track_popularity'}, inplace=True)
#     csv_data = csv_data[['id', 'acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence', 'track_popularity', 'artist_popularity', 'year', 'hit']]
    get_year = lambda dt: int(dt.split('-')[0])
    csv_data['year'] = csv_data['year'].apply(get_year)
    csv_data.to_csv(filename, index=False)
    return csv_data
    
                   
def avg_artist_popularity(artids):
    artists_pop = []
    artists = spotify.artists(artids)
    if len(artids) > 1:
        popularity = np.mean([a['popularity'] for a in artists['artists']])
    else:
        popularity = artists['artists'].pop()['popularity']

    return popularity


def flatten_dict(d):
    flat_d = {}
    for k in d:
        if type(d[k]) == dict:
            # print(d[k])
            flat_d.update(flatten_dict(d[k]))
        elif type(d[k]) == list and k == 'artists':
            artids = [art['id'] for art in d[k]]
            flat_d['artist_popularity'] = avg_artist_popularity(artids)
        else:
            flat_d[k] = d[k]
    # print(flat_d)
    return flat_d
    

In [3]:
files_17 = os.listdir('hits/')

data = []
all_ids = set()
# for file in files_17:
#     if not file.startswith('.'):
#         ids = get_pop_track_ids('2017/'+file)
#         all_ids.update(ids)

d_2016 = pd.read_csv('clean_data/2016_data.csv', skiprows=0, header=0)
d_2017 = pd.read_csv('clean_data/2017_data.csv', skiprows=0, header=0)
d_2019 = pd.read_csv('clean_data/2019_data.csv', skiprows=0, header=0)
# print(d_2016)
all_ids.update(d_2019['id'])
all_ids.update(d_2017['id'])
all_ids.update(d_2016['id'])


In [4]:
d_no_hit = pd.read_csv('clean_data/no_hits_data.csv', skiprows=0, header=0)
d_no_hits = d_no_hit[d_no_hit.year > 2015]
print(d_no_hits.columns)
print(len(d_no_hits))
d_no_hits.to_csv('clean_data/no_hits_data.csv', index=False)

Index(['id', 'acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'valence', 'track_popularity',
       'artist_popularity', 'year', 'hit'],
      dtype='object')
1414
